In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/automobile-images-dataset/white_ship/th (22).jpg
/kaggle/input/automobile-images-dataset/white_ship/th (49).jpg
/kaggle/input/automobile-images-dataset/white_ship/th (19).jpg
/kaggle/input/automobile-images-dataset/white_ship/th (13).jpg
/kaggle/input/automobile-images-dataset/white_ship/th (6).jpg
/kaggle/input/automobile-images-dataset/white_ship/th (33).jpg
/kaggle/input/automobile-images-dataset/white_ship/th (18).jpg
/kaggle/input/automobile-images-dataset/white_ship/th.jpg
/kaggle/input/automobile-images-dataset/white_ship/th (4).jpg
/kaggle/input/automobile-images-dataset/white_ship/th (10).jpg
/kaggle/input/automobile-images-dataset/white_ship/th (42).jpg
/kaggle/input/automobile-images-dataset/white_ship/th (8).jpg
/kaggle/input/automobile-images-dataset/white_ship/th (15).jpg
/kaggle/input/automobile-images-dataset/white_ship/th (44).jpg
/kaggle/input/automobile-images-dataset/white_ship/th (34).jpg
/kaggle/input/automobile-images-dataset/white_ship/th (21).jpg


In [7]:
import os
import numpy as np
import tensorflow as tf
import cv2
import tensorflow_hub as hub

In [9]:
# Parameters
data_dir = '/kaggle/input/automobile-images-dataset'
img_size = 128
list_of_data = []

In [10]:
# Loading and Preprocessing Images
for fol in os.listdir(data_dir):
    for file in os.listdir(os.path.join(data_dir, fol)):
        json_dict = {}
        img_arr = cv2.imread(os.path.join(data_dir, fol, file))[..., ::-1]  # convert BGR to RGB format
        resized_arr = cv2.resize(img_arr, (img_size, img_size))  # Reshaping images to preferred size
        json_dict['image'] = resized_arr
        json_dict['class_1'] = fol.split('_')[0]
        json_dict['class_2'] = fol.split('_')[1]
        list_of_data.append(json_dict)

In [11]:
# Splitting into x_train and y_train
x_train = []
y_train = []

for data in list_of_data:
    x_train.append(data['image'])
    y_train.append([data['class_1'], data['class_2']])

x_train = np.array(x_train) / 255
x_train = x_train.reshape(x_train.shape[0], img_size, img_size, 3)

In [12]:
# Data Augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
)
datagen.fit(x_train)

In [13]:
# Preparing labels
y_train = np.array(y_train)
class_1 = y_train[:, 0]
class_2 = y_train[:, 1]
c_1 = len(set(class_1))
c_2 = len(set(class_2))

map_1 = {j: i for i, j in enumerate(set(class_1))}
map_2 = {j: i for i, j in enumerate(set(class_2))}

class_arr1 = np.array([map_1[x] for x in class_1])
class_arr2 = np.array([map_2[x] for x in class_2])

In [ ]:
mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
inception_v3 = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"

classifier_model = mobilenet_v2

main_branch = tf.keras.Sequential([
    hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))
])

In [20]:
# Base CNN Layers
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)

# Output branches for multitask classification
colour_branch = tf.keras.layers.Dense(c_1, activation='softmax', name='colour_output')(x)
auto_branch = tf.keras.layers.Dense(c_2, activation='softmax', name='auto_output')(x)

# Model Definition
model = tf.keras.Model(inputs=inputs, outputs=[colour_branch, auto_branch])

In [23]:
# Model Compilation
model.compile(
    optimizer='rmsprop',
    loss={
        'colour_output': 'sparse_categorical_crossentropy',
        'auto_output': 'sparse_categorical_crossentropy'
    },
    loss_weights={
        'colour_output': 0.001,
        'auto_output': 1.0
    },
    metrics={
        'colour_output': ['accuracy'],
        'auto_output': ['accuracy']
    }
)

In [24]:
# Model Training
model.fit(
    {'main_input': x_train},
    {'colour_output': class_arr1, 'auto_output': class_arr2},
    epochs=10,
    batch_size=128,
    verbose=1,
    validation_split=0.2
)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - auto_output_accuracy: 0.1565 - colour_output_accuracy: 0.1700 - loss: 2.7580 - val_auto_output_accuracy: 0.2474 - val_colour_output_accuracy: 0.4579 - val_loss: 1.7330
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - auto_output_accuracy: 0.3267 - colour_output_accuracy: 0.2160 - loss: 1.7450 - val_auto_output_accuracy: 0.2684 - val_colour_output_accuracy: 0.4579 - val_loss: 2.2266
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - auto_output_accuracy: 0.2967 - colour_output_accuracy: 0.2497 - loss: 1.6559 - val_auto_output_accuracy: 0.2632 - val_colour_output_accuracy: 0.4579 - val_loss: 1.8801
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - auto_output_accuracy: 0.3851 - colour_output_accuracy: 0.2352 - loss: 1.4740 - val_auto_output_accuracy: 0.0737 - val_colour_output_accuracy: 0.3000 - val_loss: 2.5521
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - auto_output_accuracy: 0.4978 - colour_output_accuracy: 0.2578 - loss: 1.3835 -